# Model with character recognition - hidden features and characters

Builds on `RNN-Morse-features` or `RNN-Morse-full` and adds character features. There is one label per expected character (must be within the detected alphabet). 

In this two step approach a first layer deals with a reduced set of features recognition. You do not train it specifically on dits, dahs and silences you let it discover things (therefore this is "hidden"). Then this data feeds a second layer that does the character recognition.

This obviously does not work

## Create string

Each character in the alphabet should happen a large enough number of times. As a rule of thumb we will take some multiple of the number of characters in the alphabet. If the multiplier is large enough the probability of each character appearance will be even over the alphabet. 

In [ ]:
import MorseGen

morse_gen = MorseGen.Morse()
alphabet = morse_gen.alphabet14
print(132/len(alphabet))

morsestr = MorseGen.get_morse_str(nchars=132*5, nwords=27*5, chars=alphabet)
print(alphabet)
print(len(morsestr), morsestr)

## Generate dataframe and extract envelope

In [ ]:
Fs = 8000
samples_per_dit = morse_gen.nb_samples_per_dit(Fs, 13)
n_prev = int((samples_per_dit/128)*12*2) + 1
print(f'Samples per dit at {Fs} Hz is {samples_per_dit}. Decimation is {samples_per_dit/128:.2f}. Look back is {n_prev}.')
label_df = morse_gen.encode_df_decim_str(morsestr, samples_per_dit, 128, alphabet)
env = label_df['env'].to_numpy()
print(type(env), len(env))

In [ ]:
import numpy as np

def get_new_data(morse_gen, SNR_dB=-23, nchars=132, nwords=27, phrase=None, alphabet="ABC"):
    if not phrase:
        phrase = MorseGen.get_morse_str(nchars=nchars, nwords=nwords, chars=alphabet)
    print(len(phrase), phrase)
    Fs = 8000
    samples_per_dit = morse_gen.nb_samples_per_dit(Fs, 13)
    n_prev = int((samples_per_dit/128)*27) + 1 # number of samples to look back is slightly more than a dit-dah and a word space (2+3+7=12)
    print(f'Samples per dit at {Fs} Hz is {samples_per_dit}. Decimation is {samples_per_dit/128:.2f}. Look back is {n_prev}.')
    label_df = morse_gen.encode_df_decim_str(phrase, samples_per_dit, 128, alphabet)
    # extract the envelope
    envelope = label_df['env'].to_numpy()
    # remove the envelope
    label_df.drop(columns=['env','dit','dah','ele','chr','wrd'], inplace=True)
    SNR_linear = 10.0**(SNR_dB/10.0)
    SNR_linear *= 256 # Apply original FFT
    print(f'Resulting SNR for original {SNR_dB} dB is {(10.0 * np.log10(SNR_linear)):.2f} dB')
    t = np.linspace(0, len(envelope)-1, len(envelope))
    power = np.sum(envelope**2)/len(envelope)
    noise_power = power/SNR_linear
    noise = np.sqrt(noise_power)*np.random.normal(0, 1, len(envelope))
    # noise = butter_lowpass_filter(raw_noise, 0.9, 3) # Noise is also filtered in the original setup from audio. This empirically simulates it
    signal = envelope + noise
    return envelope, signal, label_df, n_prev

Try it...

In [ ]:
import matplotlib.pyplot as plt 

envelope, signal, label_df, n_prev = get_new_data(morse_gen, SNR_dB=-17, phrase=morsestr, alphabet=alphabet)

# Show
print(n_prev)
print(type(signal), signal.shape)
print(type(label_df), label_df.shape)
    
x0 = 0
x1 = 1500

plt.figure(figsize=(50,3+0.5*len(morse_gen.alphabet)))
plt.plot(signal[x0:x1]*0.5, label="sig")
plt.plot(envelope[x0:x1]*0.9, label='env')
for i, a in enumerate(alphabet):
    plt.plot(label_df[x0:x1][a]*0.9 + 1.0 + i, label=a)
plt.title("signal and labels")
plt.legend()
plt.grid()

## Create data loader
### Define dataset

In [ ]:
import torch

class MorsekeyingDataset(torch.utils.data.Dataset):
    def __init__(self, morse_gen, device, SNR_dB=-23, nchars=132, nwords=27, phrase=None, alphabet="ABC"):
        self.envelope, self.signal, self.label_df, self.seq_len = get_new_data(morse_gen, SNR_dB=SNR_dB, phrase=phrase, alphabet=alphabet)
        self.X = torch.FloatTensor(self.signal).to(device)
        self.y = torch.FloatTensor(self.label_df.values).to(device)
        
    def __len__(self):
        return self.X.__len__() - self.seq_len

    def __getitem__(self, index):
        return (self.X[index:index+self.seq_len], self.y[index+self.seq_len])
    
    def get_envelope(self):
        return self.envelope
    
    def get_signal(self):
        return self.signal
    
    def get_labels(self):
        return self.label_df
    
    def get_seq_len(self):
        return self.seq_len()

### Define data loader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_dataset = MorsekeyingDataset(morse_gen, device, -13, 132*5, 27*5, morsestr, alphabet)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=False) # Batch size must be 1

In [ ]:
signal = train_dataset.get_signal()
label_df = train_dataset.get_labels()

print(type(signal), signal.shape)
print(type(label_df), label_df.shape)

x0 = 0
x1 = 1500

plt.figure(figsize=(50,3+0.5*len(morse_gen.alphabet)))
plt.plot(signal[x0:x1]*0.5, label="sig")
plt.plot(envelope[x0:x1]*0.9, label='env')
for i, a in enumerate(alphabet):
    plt.plot(label_df[x0:x1][a]*0.9 + 1.0 + i, label=a)
plt.title("signal and labels")
plt.legend()
plt.grid()

## Create model

Let's create the model now so we have an idea of its inputs and outputs

In [ ]:
import torch
import torch.nn as nn

class MorseEnvLSTM(nn.Module):
    """
    Initial implementation
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size).to(self.device),
                            torch.zeros(1, 1, self.hidden_layer_size).to(self.device))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
    def zero_hidden_cell(self):
        self.hidden_cell = (
            torch.zeros(1, 1, self.hidden_layer_size).to(device),
            torch.zeros(1, 1, self.hidden_layer_size).to(device)
        )        
    
class MorseEnvBatchedLSTM(nn.Module):
    """
    Initial implementation
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size).to(self.device),
                            torch.zeros(1, 1, self.hidden_layer_size).to(self.device))
        self.m = nn.Softmax(dim=-1)

    def forward(self, input_seq):
        #print(len(input_seq), input_seq.shape, input_seq.view(-1, 1, 1).shape)
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(-1, 1, 1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
    def zero_hidden_cell(self):
        self.hidden_cell = (
            torch.zeros(1, 1, self.hidden_layer_size).to(device),
            torch.zeros(1, 1, self.hidden_layer_size).to(device)
        ) 
        
class MorseEnvBatchedLSTMTest(nn.Module):
    """
    Initial implementation
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size).to(self.device),
                            torch.zeros(1, 1, self.hidden_layer_size).to(self.device))
        self.m = nn.Softmax(dim=-1)

    def forward(self, input_seq):
        #print(len(input_seq), input_seq.shape, input_seq.view(-1, 1, 1).shape)
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(-1, 1, 1), self.hidden_cell)
        predictions = self.linear(lstm_out)
        return predictions
    
    def zero_hidden_cell(self):
        self.hidden_cell = (
            torch.zeros(1, 1, self.hidden_layer_size).to(device),
            torch.zeros(1, 1, self.hidden_layer_size).to(device)
        ) 
        
class MorseEnvBatchedLSTMCombo(nn.Module):
    """
    Initial implementation
    """
    def __init__(self, device, input_size=1, hidden_layer1_size=8, hidden_layer2_size=20, output1_size=6, output2_size=20):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_layer1_size = hidden_layer1_size
        self.hidden_layer2_size = hidden_layer2_size
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=hidden_layer1_size)
        self.linear1 = nn.Linear(hidden_layer1_size, output1_size)
        self.hidden1_cell = (torch.zeros(1, 1, self.hidden_layer1_size).to(self.device),
                             torch.zeros(1, 1, self.hidden_layer1_size).to(self.device))
        self.lstm2 = nn.LSTM(input_size=output1_size, hidden_size=hidden_layer2_size)
        self.linear2 = nn.Linear(hidden_layer2_size, output2_size)
        self.hidden2_cell = (torch.zeros(1, 1, self.hidden_layer2_size).to(self.device),
                             torch.zeros(1, 1, self.hidden_layer2_size).to(self.device))
        self.m = nn.Softmax(dim=-1)

    def forward(self, input_seq):
        #print(len(input_seq), input_seq.shape, input_seq.view(-1, 1, 1).shape)
        lstm1_out, self.hidden1_cell = self.lstm1(input_seq.view(-1, 1, 1), self.hidden1_cell)
        lin1_out = self.linear1(lstm1_out)
        lstm2_out, self.hidden2_cell = self.lstm2(lin1_out, self.hidden2_cell)
        predictions = self.linear2(lstm2_out.view(len(input_seq), -1))
        return predictions[-1]
    
    def zero_hidden_cells(self):
        self.hidden1_cell = (
            torch.zeros(1, 1, self.hidden_layer1_size).to(device),
            torch.zeros(1, 1, self.hidden_layer1_size).to(device)
        )         
        self.hidden2_cell = (
            torch.zeros(1, 1, self.hidden_layer2_size).to(device),
            torch.zeros(1, 1, self.hidden_layer2_size).to(device)
        )         
    
class MorseEnvLSTM2(nn.Module):
    """
    LSTM stack
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6, dropout=0.2):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers=2, dropout=dropout)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(2, 1, self.hidden_layer_size).to(self.device),
                            torch.zeros(2, 1, self.hidden_layer_size).to(self.device))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
    def zero_hidden_cell(self):
        self.hidden_cell = (
            torch.zeros(2, 1, self.hidden_layer_size).to(device),
            torch.zeros(2, 1, self.hidden_layer_size).to(device)
        )        
        
class MorseEnvBatchedLSTM2(nn.Module):
    """
    LSTM stack - dataset compatible
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6, dropout=0.2):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_layer_size = hidden_layer_size
        self.output_size = output_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers=2, dropout=dropout)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(2, 1, self.hidden_layer_size).to(self.device),
                            torch.zeros(2, 1, self.hidden_layer_size).to(self.device))
        self.m = nn.Softmax(dim=-1)

    def forward(self, input_seq):
        #lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(-1, 1, 1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1] if self.output_size == 1 else self.m(predictions[-1])
    
    def zero_hidden_cell(self):
        self.hidden_cell = (
            torch.zeros(2, 1, self.hidden_layer_size).to(device),
            torch.zeros(2, 1, self.hidden_layer_size).to(device)
        )        
        
class MorseEnvNoHLSTM(nn.Module):
    """
    Do not keep hidden cell
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        h0 = torch.zeros(1, 1, self.hidden_layer_size).to(self.device)
        c0 = torch.zeros(1, 1, self.hidden_layer_size).to(self.device)
        lstm_out, _ = self.lstm(input_seq.view(len(input_seq), 1, -1), (h0, c0))
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
class MorseEnvBiLSTM(nn.Module):
    """
    Attempt Bidirectional LSTM: does not work
    """
    def __init__(self, device, input_size=1, hidden_size=12, num_layers=1, num_classes=6):
        super(MorseEnvBiLSTM, self).__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidirection
    
    def forward(self, x):
        # Set initial states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x.view(len(x), 1, -1), (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out[-1]    

Create the model instance and print the details

In [ ]:
nb_alpha_features = len(alphabet)
morse_env_model = MorseEnvBatchedLSTMCombo(
    device, hidden_layer1_size=7, hidden_layer2_size=nb_alpha_features, output1_size=5, output2_size=nb_alpha_features).to(device)
morse_env_loss_function = nn.MSELoss()
morse_env_optimizer = torch.optim.Adam(morse_env_model.parameters(), lr=0.001)

print(morse_env_model)
print(morse_env_model.device)

In [ ]:
# Input and hidden tensors are not at the same device, found input tensor at cuda:0 and hidden tensor at cpu
for m in morse_env_model.parameters():
    print(m.shape, m.device)
X_t = torch.rand(n_prev)
#X_t = torch.tensor([-0.9648, -0.9385, -0.8769, -0.8901, -0.9253, -0.8637, -0.8066, -0.8066, -0.8593, -0.9341, -1.0000, -0.9385])
X_t = X_t.cuda()
print(X_t)
morse_env_model(X_t)

In [ ]:
import torchinfo
channels=10
H=n_prev
W=1
torchinfo.summary(morse_env_model, input_size=(channels, H, W))

## Train model

In [ ]:
it = iter(train_loader)
X, y = next(it)
print(X.reshape(n_prev,1).shape, X[0].shape, y[0].shape)
print(X[0], y[0])
X, y = next(it)
print(X[0], y[0])

In [ ]:
%%time
from tqdm.notebook import tqdm

epochs = 2
morse_env_model.train()

for i in range(epochs):
    train_losses = []
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=True)
    for j, train in loop:
        X_train = train[0][0]
        y_train = train[1][0]
        morse_env_optimizer.zero_grad()
        morse_env_model.zero_hidden_cells() # this model needs to reset the hidden cells
        y_pred = morse_env_model(X_train)
        single_loss = morse_env_loss_function(y_pred, y_train)
        single_loss.backward()
        morse_env_optimizer.step()
        train_losses.append(single_loss.item())
        # update progress bar
        if j % 1000 == 0:
            loop.set_description(f"Epoch [{i+1}/{epochs}]")
            loop.set_postfix(loss=np.mean(train_losses))

print(f'final: {i+1:3} epochs loss: {np.mean(train_losses):6.4f}')

In [ ]:
save_model = True
if save_model: 
    torch.save(morse_env_model.state_dict(), 'models/morse_char_model')
else:
    morse_env_model.load_state_dict(torch.load('models/morse_char_model', map_location=device))

### Predict (test)

In [ ]:
# new phrase
morsestr_test = MorseGen.get_morse_str(nchars=132//2, nwords=27//2, chars=alphabet)
print(alphabet)
print(len(morsestr_test), morsestr_test)

In [ ]:
# Dataloader
test_dataset = MorsekeyingDataset(morse_gen, device, -10, 132*2, 27*2, morsestr_test, alphabet)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False) # Batch size must be 1

envelope = test_dataset.get_envelope()
signal = test_dataset.get_signal()
label_df = test_dataset.get_labels()

In [ ]:
%%time
p_alpha = {}
for a in alphabet:
    p_alpha[a] = []
morse_env_model.eval()

for X_test0, y_test0 in test_loader:
    X_test = X_test0[0]
    pred_val = morse_env_model(X_test).cpu()
    for i, a in enumerate(alphabet):
        p_alpha[a].append(pred_val[i].item())
        
for a in alphabet:
    p_alpha[a] = np.array(p_alpha[a])

In [ ]:
x0 = 0
x1 = 1500
sig = signal[n_prev:]
env = envelope[n_prev:]
plt.figure(figsize=(50,3+0.5*len(morse_gen.alphabet)))
plt.plot(sig[x0:x1]*0.5, label="sig")
plt.plot(env[x0:x1]*0.9, label='env')
for i, a in enumerate(alphabet):
    plt.plot(p_alpha[a][x0:x1]*0.9 + 1.0 + i, label=a)
plt.title("predictions")
plt.legend()
plt.grid()
print(alphabet, morsestr_test)

In [ ]:
plt.figure(figsize=(50,6))
plt.plot(sig[x0:x1]*0.5, label="sig")
plt.plot(env[x0:x1]*0.9, label='env')
plt.plot(p_dit[x0:x1]*1.2 + p_dah[x0:x1] + 1.0, label='s1')
plt.plot(1.0 - p_ele[x0:x1] - p_chr[x0:x1] - p_wrd[x0:x1] + 1.0, label='s2')
plt.title("reconstruction")
plt.legend()
plt.grid()